In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep # control the crawl rate to avoid hammering the servers with too many requests
from random import randint
import re # regular expression

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_binary # adds chromedriver binary to path

In [2]:
# testing on just one foundation
def parsing_pages(list_of_pages):
    pages = []
    for page in list_of_pages:
        if pd.notnull(page):
            all_pages = requests.get(page)
            each_page = BeautifulSoup(all_pages.content, "html.parser")
            pages.append(each_page)
            sleep(randint(2,6))
        else: 
            pages.append(None)
    return pages

In [3]:
# what i need to get from the reviews
# 1) stars (0 - 5) ## "data-comp="StarRating" <div aria-label="4 stars">
# 2) reviewer id ## <a href="users/Tarah84" ...> under <strong data-at="nickname">____</strong>
# 3) reviewer features ## two lines under "nickname"
# 4) purchased shade ## line after headline of the review ("<h3 class>")
driver = webdriver.Chrome()
driver.get("https://www.sephora.com/product/pro-filtr-soft-matte-longwear-foundation-P87985432?skuId=2164697")

list_of_reviews = []
items_from_page = driver.find_element_by_css_selector('.data-comp="StarRating"')





KeyboardInterrupt: 